In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://ucsdtritons.com/sports/mens-basketball/stats/'

page = requests.get(url) #requests object

soup = BeautifulSoup(page.text, 'html')

In [3]:
table = soup.find_all('table')[1]
print(table)

<table accordion-table="" class="sidearm-table collapse-on-medium accordion" sortable-table="">
<caption class="hide">Overall Individual Statistics</caption>
<thead class="sidearm-primary">
<tr>
<th class="text-left" rowspan="2" scope="col">#</th>
<th class="text-hide" rowspan="2" scope="col">Player</th>
<th rowspan="2" scope="col">GP</th>
<th rowspan="2" scope="col">GS</th>
<th colspan="2" scope="colgroup">Minutes</th>
<th colspan="3" scope="colgroup">FG</th>
<th colspan="3" scope="colgroup">3PT</th>
<th colspan="3" scope="colgroup">FT</th>
<th colspan="2" scope="colgroup">Scoring</th>
<th colspan="4" scope="colgroup">Rebounds</th>
<th rowspan="2" scope="col">PF</th>
<th rowspan="2" scope="col">AST</th>
<th rowspan="2" scope="col">TO</th>
<th rowspan="2" scope="col">STL</th>
<th rowspan="2" scope="col">BLK</th>
<th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>
</tr>
<tr>
<th scope="col">TOT</th>
<th scope="col">AVG</th>
<th scope="col">FGM</th>
<th sco

In [4]:
#ISSUE: COLUMNS ARE OUT OF ORDER
#After #, GP, GS it goes out of order
    #It jumps to 

#extract the table headers
headers = table.find_all('th', attrs={"scope": "col"})
print(headers)
num_headers = len(headers)

#loop through the headers
table_headers = [title.text.strip() for title in headers]

#Move these headers to the back of the list to fix the order
headers_to_move = ["PF", "AST", "AST/G", "TO", "STL", "BLK", "Bio Link"]

fixed_headers = [h for h in table_headers if h not in headers_to_move]  # Keep other headers in original order
moved_headers = [h for h in table_headers if h in headers_to_move]  # Move these to the end

# Combine lists to get the final order
final_headers = fixed_headers + moved_headers
print(final_headers)

[<th class="text-left" rowspan="2" scope="col">#</th>, <th class="text-hide" rowspan="2" scope="col">Player</th>, <th rowspan="2" scope="col">GP</th>, <th rowspan="2" scope="col">GS</th>, <th rowspan="2" scope="col">PF</th>, <th rowspan="2" scope="col">AST</th>, <th rowspan="2" scope="col">TO</th>, <th rowspan="2" scope="col">STL</th>, <th rowspan="2" scope="col">BLK</th>, <th aria-hidden="true" class="hide-on-large" rowspan="2" scope="col">Bio Link</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>, <th scope="col">FGM</th>, <th scope="col">FGA</th>, <th scope="col">FG%</th>, <th scope="col">3PT</th>, <th scope="col">3PTA</th>, <th scope="col">3PT%</th>, <th scope="col">FTM</th>, <th scope="col">FTA</th>, <th scope="col">FT%</th>, <th scope="col">PTS</th>, <th scope="col">AVG</th>, <th scope="col">OFF</th>, <th scope="col">DEF</th>, <th scope="col">TOT</th>, <th scope="col">AVG</th>]
['#', 'Player', 'GP', 'GS', 'TOT', 'AVG', 'FGM', 'FGA', 'FG%', '3PT', '3PTA', '3PT%', 'FTM', 'FT

In [5]:
#Extract data for columns
column_data = table.find_all('tr')
df = pd.DataFrame(columns = final_headers)

for row in column_data[2:]: 
    row_data = row.find_all('td')
    individual_row_data = [data.get_text(separator = " ").strip() for data in row_data]
    individual_row_data[1] = " ".join(individual_row_data[1].split()).split(" ", 1)[-1]
    if len(individual_row_data) > 1:
        words = individual_row_data[1].split()  # Split into words
    if len(words) > 2:  
        individual_row_data[1] = " ".join(words[2:])  # Keep everything after the first two words
        
    print(individual_row_data)
    
    length = len(df)
    df.loc[length] = individual_row_data


['12', 'Tait-Jones, Aniwaniwa', '26', '26', '775', '29.8', '163', '283', '.576', '13', '37', '.351', '176', '223', '.789', '515', '19.8', '30', '102', '132', '5.1', '52', '88', '53', '31', '10', 'View Bio']
['13', 'McGhie, Tyler', '26', '26', '783', '30.1', '154', '346', '.445', '87', '217', '.401', '41', '47', '.872', '436', '16.8', '7', '96', '103', '4.0', '39', '45', '20', '26', '6', 'View Bio']
['03', 'Gray, Hayden', '26', '26', '811', '31.2', '107', '222', '.482', '43', '102', '.422', '32', '47', '.681', '289', '11.1', '17', '63', '80', '3.1', '45', '84', '39', '90', '6', 'View Bio']
['24', 'Kapic, Nordin', '25', '24', '607', '24.3', '83', '216', '.384', '38', '126', '.302', '55', '76', '.724', '259', '10.4', '35', '94', '129', '5.2', '64', '30', '37', '23', '10', 'View Bio']
['05', 'Rochelin, Justin', '26', '0', '501', '19.3', '60', '134', '.448', '26', '81', '.321', '19', '28', '.679', '165', '6.3', '20', '66', '86', '3.3', '75', '20', '20', '14', '8', 'View Bio']
['08', 'Howell

In [7]:
df

#Export this into a csv
df.to_csv(r'/Users/ryanuyeki/Downloads/asa/ucsd_player_stats.csv', index=False, encoding = "utf-8")